의존성 설치

In [6]:
%%capture --no-stderr
%pip install langchain_community langchainhub chromadb langchain langgraph tavily-python langchain-text-splitters langchain_openai

In [ ]:
%pip install streamlit

In [51]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

tavily 설정

In [ ]:
from tavily import TavilyClient
tavily = TavilyClient(api_key='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')

# 예시
response = tavily.search(query="에어컨은 누가 만들었어?", max_results=3)
context = [{"url": obj["url"], "content": obj["content"]} for obj in response['results']]

context


gpt-4o-mini 준비

In [8]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0)

벡터저장소에 담을 예제문서

In [ ]:
### Index

from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
# docs>sublist>item
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding = OpenAIEmbeddings(model="text-embedding-3-small")
)
retriever = vectorstore.as_retriever()

관련성 체커

In [10]:
### Relevance Checker

from langchain_core.output_parsers import JsonOutputParser

system = """You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n document: {document} "),
    ]
)

retrieval_grader = prompt | llm | JsonOutputParser()

llm 응답 생성기

In [65]:
### Generate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    context를 참고할 땐 그 출처를 꼭 명시해. 출처는 metatdata.source를 확인하면 돼.
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n context: {context} "),
    ]
)

# Chain
rag_chain = prompt | llm | StrOutputParser()

허상 체크

In [12]:
### Hallucination Grader

system = """You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    single key 'score' and no preamble or explanation."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "documents: {documents}\n\n answer: {generation} "),
    ]
)

hallucination_grader = prompt | llm | JsonOutputParser()


엣지 노드 구성

In [62]:
from pprint import pprint
from typing import List

from langchain_core.documents import Document
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph

### State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]
    retry: int


### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    print(question)
    print(documents)
    return {"documents": documents, "question": question, "retry": 0}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    retry = state["retry"]
    return {"documents": documents, "question": question, "generation": generation, "retry": retry + 1}

def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    print(state)
    question = state["question"]

    # Web search
    docs = tavily.search(query=question)['results']
    web_results = []
    for d in docs:
      wd = Document(page_content=d["content"], metadata={"source": d["url"]})
      web_results.append(wd)

    print("########## web_results")
    print(web_results)

    retry = state["retry"]
    return {"documents": web_results, "question": question, "retry": retry + 1}

### Conditional edge
def relevance_checker(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---RELEVANCE CHECKER---")
    question = state["question"]
    documents = state["documents"]
    retry = state["retry"]

    if retry > 1:
      raise ValueError("failed: not relevant")

    # Score each doc
    filtered_docs = []
    web_search = "No"

    for d in documents:
      result = retrieval_grader.invoke(
          {"question": question, "document": d.page_content}
      )
      score = result["score"]

      if score.lower() == "yes":
          print("---GRADE: DOCUMENT RELEVANT---")
          filtered_docs.append(d)
      else:
          print("---GRADE: DOCUMENT NOT RELEVANT---")

    print("########### filtered_docs")
    print(filtered_docs)
    if len(filtered_docs) > 0:
      return "generate"
    else:
      return "websearch"

def hallucination_checker(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    retry = state["retry"]

    if retry > 2:
      raise ValueError("failed: hallucination")

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION ADDRESSES QUESTION---")
        return "useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"



그래프 생성

In [63]:
# Build graph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("generate", generate)  # generatae
workflow.add_node("websearch", web_search)  # web search

workflow.set_entry_point("retrieve")

workflow.add_conditional_edges(
    "retrieve",
    relevance_checker,
    {
        "generate": "generate",
        "websearch": "websearch"
    },
)

workflow.add_conditional_edges(
    "websearch",
    relevance_checker,
    {
        "generate": "generate",
        "websearch": "websearch"
    },
)

workflow.add_conditional_edges(
    "generate",
    hallucination_checker,
    {
        "useful": END,
        "not supported": "generate"
    },
)

컴파일 및 실행

In [67]:
# Compile
app = workflow.compile()

# Test

inputs = {"question": "카카오 캠퍼스는 어디에있어?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
카카오 캠퍼스는 어디에있어?
[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nS

In [70]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com


34.139.209.90
⠙your url is: https://gentle-bears-share.loca.lt


In [ ]:
!streamlit run /content/app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.139.209.90
⠙


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.209.90:8501

your url is: https://tidy-forks-juggle.loca.lt
USER_AGENT environment variable not set, consider setting it to identify your requests.
